# **Milestone 5: Trade-offs and Interactions of Bias and Discrimination with other verticals in Trustworthy AI**
# **PRACTICE NOTEBOOK - Explainability**

In this pratice notebook, we explore three explainability methods in the context of the hiring algorithm example. As a reminder, a company is looking to hire a new employee. They use a machine learning algorithm to select the top candidates. The candidates are assigned either 0 if they're not selected or 1 if they are. 

We will explore the three following methods:
- Permutation Feature Importance
- Shapley Values
- LIME (Local Interpretable Model-agnostic Explanations)

We use the same data as in Milestone 3 and 4, but we consider candidates from the *Black* and *White* groups only. We consider that fairness is achieved if the Disparate Impact metric is between 0.8 ans 1.2.

As a side note, we use a lightGBM classifier in this notebook from the lightgbm package. The results reported here are valid when using lightgbm version 3.3.0, but we have noticed they may differ when using other versions. If in colab, there is a line of code installing the right version to start with. 

# 1- Setting the problem

In this section, there is nothing to do, we:
- import modules, define some functions, get raw data and pre-process it
- split data into training/testing, define a model and train it
- get model accuracy and disparate impact 

## 1.1. Import modules, define some functions, get raw data and pre-process it

In [ ]:
!pip install lightgbm==3.3.0

In [ ]:
# ======================= MODULES ======================
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeClassifier
import lightgbm
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import seaborn as sns
from tqdm.notebook import tqdm

In [ ]:
# WARNING: Only run if running on Google Colab
# ======================= LOAD DATA ======================
!pip3 install pickle5
import pickle5 as pickle
!gdown --id 1-Wd1evAoDs4YsjRLfC-ifarmQL-Ozg3R # download data file from public link and place it in content/ folder

In [ ]:
# ======================= LOAD DATA ======================
'''
Uncomment the code below if you are running this from your local machine
Note: Place data.pickle file in the same folder as the .ipynb notebook (Download link: https://docs.google.com/uc?export=download&id=1-Wd1evAoDs4YsjRLfC-ifarmQL-Ozg)
'''
# with open('data.pickle', 'rb') as handle:
#     data = pickle.load(handle)   
if 'google.colab' in str(get_ipython()): # if running from colab
  with open('/content/data.pickle', 'rb') as handle:
      data = pickle.load(handle)  
data[:5] #display the first 5 candidates data

In [ ]:
# ==================FUNCTION========================
def split_data_from_df(data):
  y = data['Label'].values
  colX = list(data.columns[(data.columns != 'Label') & (data.columns != 'Ethnicity')])
  X = data[colX].values
  dem = data['Ethnicity'].copy()
  return X,y,dem
# ==================PRE-PROCESSING========================
# we filter only White/Black candidate and remove all nan values
data = data.dropna()
data = data[(data['Ethnicity']=='White') | (data['Ethnicity']=='Black')].reset_index(drop=True)
data = data.drop(columns=['Gender'])
enc = LabelEncoder()
data['Ethnicity'] = enc.fit_transform(data['Ethnicity'])
print(enc.transform(['White','Black']))
data[:2]

## 1.2. Split data into train/test set and train the model

We now have the dataframe with only Black (0) and White (1) candidates left, their labels (Fail:0 or Pass:1) and their 500 features. Here we define a model and train it on a training set. Note that we use here a different model than in the previous sections (*LGBMClassifier* instead of *RidgeClassifier*) as the Shapley value pakage we will use in the next section does not support yet the *RidgeClassifier* model. 

In [ ]:
# split data in train/test
data_train, data_test = train_test_split(data,test_size = 0.3,random_state=42)
# get X,y, gender, ethnicity
X,y,dem = split_data_from_df(data)
X_train,y_train,dem_train = split_data_from_df(data_train)
X_test,y_test,dem_test = split_data_from_df(data_test)
# define and train model
model = LGBMClassifier(random_state=40)
model.fit(X_train,y_train)

## 1.3. Evaluate model accuracy and disparate impact

We will use the disparate impact metric as a measure of fairness in this notebook. We implement the function *get_disparate_impact* that calculate disparate impact between *Black* and *White* candidates on the predictions *y_pred* gicen the ethnicity of the candidates. 

In [ ]:
# define a function that gets the disparate impact
def success_rate(y_pred,idx_group):
    '''gets the success rate of a group given:
        y_pred: (n_pred,). np array containting the predictions for all candidates (0 or 1)
        idx_group: Indexes of the candidates belonging to the group of interest 
    '''
    y_pred_group = y_pred[idx_group]
    sr = (y_pred_group == 1).sum() / len(y_pred_group)
    return sr
def get_disparate_impact(y_pred,e):
    '''Calculate disparate impact using predicted outcome. 
    Inputs:
        y_pred : (n_pred,). np array containting the predictions for each candidate (0 or 1)
        e: (n_pred,). np array containing the ethnicity of each candidate (Black:0 and White:1)
    Output: Disparate impact
    '''
    # indexes corresponding to black/white candidates
    idx_b = np.where(e == 0)[0] #black
    idx_w = np.where(e == 1)[0] #white
    # success rates of black/white group
    sr_w = success_rate(y_pred,idx_w)
    sr_b = success_rate(y_pred,idx_b)
    return sr_b/sr_w

We evaluate the accuracy of our trained model on the test set, and using the function defined above, calculate the disparate impact.

In [ ]:
# evaluate accuracy on test
y_pred_test = model.predict(X_test)
acc = accuracy_score(y_test,y_pred_test)
print("Accuracy is %.2f"%acc)
# get ethnicity for test set
X_test,y_test,dem_test = split_data_from_df(data_test)
e_test = np.array(dem_test)
# get disparate impact on test set predictions
di = get_disparate_impact(y_pred_test,e_test)
print("Disparate impact is %.2f"%di)

We get a Disparate impact of 0.76 and an accuracy of 0.70. In the next three sections, we will try to understand what particular features are driving these results (both the accuracy and disparate impact).

Note that these results are valid for lightgbm version 3.3.0 and that we've noticed some differences when using other versions. 

# 2. Permutation feature importance

In this part, we demonstrate how we will see how to use permutation feature importance in the context of this problem, and how we can apply it to fairness.

As a reminder, we copy here the information on Feature Importance from the Supporting Notebook. 


Feature Importance looks to assign a score to each feature relative to its importance in the prediction. There are different methods to perform feature importance depending on the model used, as described in Jason Brownlee's article [How to Calculate Feature Importance With Python](https://machinelearningmastery.com/calculate-feature-importance-with-python/).  For instance, in a simple linear regression, the feature importance can simply be the weight times the feature value. For some more complicated or non-linear models, other model-agnostic methods are needed. One is *Permutation Feature Importance*. Let's call $X$ the feature matrix (each row i is a sample and each column j is a feature) and $e$ the error of the model when predicting for $X$. We calculate the feature imporance of feature j by:
1. Create feature matrix $X_j$ by permuting column j
2. Calculate the new error $e_j$ when predicting for $X_j$ 
3. The feature importance is $FI_j = e_j - e$. 

In other words, we randomly permute the values of one of the features and see how this impact the results. 

LGBMClassifierIn the following, we replicate the code above: we train a LGBMClassifier on a training set and evaluate the base accuracy and disparate impact on the test set. 

In [ ]:
# define model
model = LGBMClassifier(random_state = 40)
# split data in train/test
data_train, data_test = train_test_split(data,test_size = 0.3,random_state=42)
# get X,y,ethnicity
X,y,dem = split_data_from_df(data)
X_train,y_train,dem_train = split_data_from_df(data_train)
X_test,y_test,dem_test = split_data_from_df(data_test)
# train
model.fit(X_train,y_train)
# Get base accuracy and feature importance
y_pred_test = model.predict(X_test)
acc_base = accuracy_score(y_test,y_pred_test)
print("Accuracy is %.2f"%acc)
e_test = np.array(dem_test)
di_base = get_disparate_impact(y_pred_test,e_test)
print("Disparate impact is %.2f"%di)

**Instructions:** Create a function *permute_X* that takes as an input a matrix X and the index j of a column, and that returns the same matrix but with column j randomly shuffled.

In [ ]:
def permute_X(X,j):
    Xj = X.copy()
    ### Do something###
    return Xj

**Instructions:** We want to permute each feature in isolation and see the impact this has on the model's prediction. We do this on the test dataset so you will permute the columns of *X_test* in this part. Also, we want to repeat the experiment a number of times (10 in this example), so we can quantify the impact of a feature using an average rather than a single number. The steps are as follow:

Repeat the following for 10 iterations (*n_iter=10*):
For each feature (j from 0 to 499):
- permute column j and get the permuted input matrix (using the pre-defined function)
- get the predictions of the model on this permuted input matrix
- calculate the accuracy and disparate impact using those predictions
store the disparate impact and accuracy in 2 numpy array of size (n_iter,n_features). 

The skeleton of the code is available below. Note that this may take a few minutes to compile. 

In [ ]:
np.random.seed(10)
n_features = 500
n_iter = 10
accs = np.zeros((n_iter,n_features))
dis = np.zeros((n_iter,n_features))
for j in tqdm(range(n_features)):
    for i in range(n_iter):
        ### DO SOMETHING ###
        continue

Because we are interested in the difference of these accuracy/disparate impact values with the base values (to isolate the impact of permuting a feature), we get numpy array with these differences directly: 

In [ ]:
# difference with base
accs_diff = accs - acc_base
dis_diff = dis - di_base

### 2.1. Original permutation feature importance
The original permutation feature importance process looks at the difference in errors, which we measure here using the accuracy. 

**Instructions:** For each feature, 
- get the mean accuracy differences and standard deviations over the 10 iterations. You should end up with an array of size (500,) for the means and same for the stds (one value for each feature). We refer to these arrays as *means* and *stds*. 
- In the *means* array, negative values represent features for which, when randomly permuted, accuracy decreases. These feature therefore have the most positive impact on accuracy. On the opposite, positive values represent features that make the accuracy decrease. 
- Sort the *means* and *stds* arrays by ascending *means* values. Use argsort to get the indexes in ascending order. Note that the indexes correspond to the feature we are looking at (features 0 to 499).
- Plot the means and stds ot the 30 features with the most positive impact (with the highest negative values) and the 30 features with the most negative impact (with the highest positive values) and print to which feature they correspond to. You can use the [*plt.errorbar*](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.errorbar.html) function to plot the mean and std together.

You should get the following graph:
<center><img src="https://drive.google.com/uc?id=1rVxJRr_cu4PyXArsqHrdeZPRHJBVdJmQ"></center>

========indexes of 30 highest positive impact features=========<br>
[  1   3  28  69 126 481 139 259 320 421  91 164 343   4 411 319   5 238
 123   0 388 394  95 190  85 473 424   7 341 245]<br>
========indexes of 30 highest negative impact features=========<br>
[260 173  30 393  45 449  12  90 489 103 490 243 159 296  32 417 315 383
 256 223   2 221  61 269 385 207 439  35 318  37]<br>


Using this approach, it looks like feature 1 and 3 (when features are numbered from 0 to 499) are the most impactful on the accuracy: randomly shuffling these bring the accuray down by a lot. 

### 2.2. Permutation feature importance using disparate impact

Now we directly want to see the effect of permuting features on the disparate impact metric. 

**Instructions:** Repeat the steps for plotting effect on accuracy but using disparate impact instead. 

You should now get the following graph:
<center><img src="https://drive.google.com/uc?id=1q9OywHiOToeeAByrpcHz9bjL5ZZoF1Sp"></center>

========indexes of 30 highest positive impact features========= <br>
[247 218 126 155 190  28 448 107 162  32  22 317 205 473  13  11 470 321
 217  99 303 158 371 163 499 331 357  44 182 413] <br>
========indexes of 30 highest negative impact features========= <br>
[  3   1   2 184  15  12 195  50 127  31 404   4  76   8 129  97 443 216
 139 465 119 113 345 270 123  48 268 203  23 397]

Again, feature 3 and 1 look like they have the biggest impact. In the following section, we will try to remove feature 3 before training the model to see the effect this will have on the disparate impact. According to this, it should increase. 

### 2.3. Re-training the model by removing feature 3

In [ ]:
# define model
model = LGBMClassifier(random_state = 40)
# split data in train/test
data_train, data_test = train_test_split(data,test_size = 0.3,random_state=42)

**Instructions:** Retrain the model but removing feature 3.
- copy the dataframes data_train and data_test from above, and remove feature 3 (column name is a numerical 3)
- get X_train and X_test from these new dataframes using the *split_data_from_df* function.
- train the model on X_train
- Predict on X_test, and evaluate the accuracy and disparate impact 

You should get a new accuracy of 0.68 and disparate impact of 0.84. Is this what you expected to happen? 

# 3. SHAPLEY values

The concept of shapley values is explained in the theory notebook (supporting notebook). In a sentence, the shapley value is: "the average marginal contribution of an instance of a feature among all possible coalitions".




In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
# define model
model = LGBMClassifier(random_state = 40)
# split data in train/test
data_train, data_test = train_test_split(data,test_size = 0.3,random_state=42)
# get X,y,ethnicity for trai#n and test sets
X,y,dem = split_data_from_df(data)
X_train,y_train,dem_train = split_data_from_df(data_train)
X_test,y_test,dem_test = split_data_from_df(data_test)
# train model
model.fit(X_train,y_train)

**Instructions: Follow this tutorial to:**
https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/tree_based_models/Census%20income%20classification%20with%20LightGBM.html
- **build a tree explainer object for the model** (cell 5 in the link)
- **get the shapley values for the features (for the entire dataset X)** (cell 5 in the link)
- **plot the summary plot** (cell 8 in the link)

Note: this can be done in 3 lines of code. 

**What are the 3 most influencial features for the pass/fail decision?**

Note that because there are 2 classes, there are 2 sets of shapley values. This is redundant information as there are only 2 outputs, 0 or 1 so the factors that are moving an outputs towards 1 are logically taking it away from 0 with equal contribution. We therefore only focus on the shapley values relative to 1, i.e. what features are bringing a candidate closer to or further away from success?

**Instructions: Re-plot the summary plot but of *shap_values[1]*, i.e. only the second set of shapley values** A different type of graph should be plotted.

Instead of a bar graph showing the mean absolute shap value, you should now get the following graph:
<center><img src="https://drive.google.com/uc?id=1bD32ZAFQ0ZElMRyRLxoMry4nZqf1k0wy"></center>

This shows the impact on model output of each observations (each dot representing the marginal contribution of our feature when added to some coalition of features). For instance for feature 1, the blue dots represent low feature values for feature 1 that correlate with a negative impact on the model output (i.e low feature values for feature 1 correlate with lower success for candidates).

The SHAP values represent the impact of a feature on model output. Because we are in a classification setting, the outputs are either 0 (fail) or 1 (pass). Hence a value of 0.3 for instance will mean that 70% of the time the outcome is 0 and 30% of the time it is 1. 


Finally, we can look at the features impacting the most a specific decision (candidate i) using: *shap.plots.force(explainer.expected_value[1],shap_values[1][i])*. Note that here the subsript [1] refers to the second set of shapley values as we explained a few paragraphs above. 

**Instructions: Plot the force plot for candidate 3**. Note, you might need to run *shap.initjs()* before for visualisation of the output. 

You should get this graph: 

<center><img src="https://drive.google.com/uc?id=1xZIdi620LIdgJLOiKebrdhPvDoj0ltNR"></center>

Note that the results are here in "raw score". Indeed, lightGBM main output is a raw score (here approximately between -4 and 2) as shown below. This raw score is then transformed by the algorithm in probability of outcome. The base value (given by explainer.expected_value[1]) is the mean output for all candidates (in terms of raw score). Then the red features show the feature that push the result upward for that candidate (Feature 1 mainly) and the features that push results downwards (mainly Feature 3) compared to the base value. The final result of 1.32 is the sum of all the features' contributions and is the raw score for that candidate. 

In [ ]:
#raw score output
y_pred_train_raw = model.predict(X_train,raw_score = True) # raw_score
print("Raw scores ->", y_pred_train_raw)
#  Base value verification
print("Base value is %.4f" %y_pred_train_raw.mean())
# raw score of candidate 3
y_pred_raw = model.predict(X,raw_score = True) # raw_score
print("Row score of candidate 3 = %.2f" %y_pred_raw[3])

### Interaction with fairness

How can we use the SHAP values for fairness? Intuitively, we can look at the shap values for different groups and calculate the difference. For features with a significant difference in SHAP values, it means that the model is not using these features in the same way for different groups and that they might be causing a bias. This link (https://shap.readthedocs.io/en/latest/example_notebooks/overviews/Explaining%20quantitative%20measures%20of%20fairness.html) explains how one can use SHAP values for fairness. 

In [ ]:
group_mask = (data['Ethnicity']==0)

**Following the link, can you plot the *group_difference_plot* for White and Black? Use the argument *max_display=10* for readability.**


**What feature most impact Black and White in different ways?** Are these results consistent with the results from Permutation Feature Importance?

# 4. LIME (Local Interpretable Model-agnostic Explanations)

In this final section, we will look at the explanation method LIME (Local Interpretable Model-agnostic Explanations).

We do the following:

- Import lime and lime_tabular
- Define a LGBMClassifier and train it on the train set
- Evaluate the accuracy and disparate impact

In [ ]:
!pip install lime

In [ ]:
import lime
import lime.lime_tabular

In [ ]:
# define model
model = LGBMClassifier(random_state = 40)
# split data in train/test
data_train, data_test = train_test_split(data,test_size = 0.3,random_state=42)
# get X,y, gender, ethnicity
X,y,dem = split_data_from_df(data)
X_train,y_train,dem_train = split_data_from_df(data_train)
X_test,y_test,dem_test = split_data_from_df(data_test)
# train
model.fit(X_train,y_train)
# Get base accuracy and feature importance
y_pred_test = model.predict(X_test)
acc_base = accuracy_score(y_test,y_pred_test)
print("Accuracy is %.2f"%acc_base)
di_base = get_disparate_impact(y_pred_test,dem_test)
print("Disparate impact is %.2f"%di_base)

**Instructions: Create an explainer object using *lime.lime_tabular.LimeTabularExplainer*** (refer to https://lime-ml.readthedocs.io/en/latest/lime.html#module-lime.lime_tabular)

**Instructions: Explain instances 0,3,56 of the test:**
- Use *explainer.explain_instance* to create an explanation object *exp* (refer to https://lime-ml.readthedocs.io/en/latest/lime.html#lime.lime_tabular.LimeTabularExplainer.explain_instance)
- Use the argument *num_features = 6* to limit the explaination to 6 features only
- display using the command *exp.show_in_notebook(show_all=False)*

Note: This might take a while to run. 

For instance 3 (candidate 3), you should get the followin figure: 
<center><img src="https://drive.google.com/uc?id=1OxAS4JD5lD6Oh_T82VHV8eoL3Meq-xP8"></center>

There are 3 parts of the graph:
- On the left is the predicted probability that this example is classified as Fail(0) or Pass(1). 
- In the middle, the 6 features with the most important influence on the result are presented. In orange are the ones that push towards an outcome of 1, and in blue towards an outcome of 0. The bars and float numbers associated present the relative importance of these features. For instance, Feature 3 contributes 0.11 towards a prediction of 1. Next to this bar is the name of the feature concerned (here 3) and its value range (<= -1.36). Indeed, continuous features are discretized. The bin of values in which the feature falls decides of the influence of the feature. For example, candidates 0 and 56 of the test set both have feature 3 values between -1.36 and 0.01, which corresponds to a contribution of +0.05 towards outcome 1 instead od +0.11 for candidate 3.
- On the right, the actual feature value for the candidate is shown. 

Again, using LIME for local explanations, one can see that features 1 and 3 seem to be very important. 
